## Analysis of Past Loseit Challenges

In [190]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import praw
import datetime as dt
import markdown
from lxml import etree
import lxml
import gspread
import re
import matplotlib.pyplot as plt
import seaborn as sns
from oauth2client.service_account import ServiceAccountCredentials
from pathlib import Path
import os

Create the Data Directories

In [192]:
if not os.path.exists('./data/raw_data/'):
    os.makedirs('./data/raw_data/')
if not os.path.exists('./figures/'):
    os.makedirs('./figures/')

In [ ]:
reddit = praw.Reddit('loseit_data')

In [ ]:
loseit_sub = reddit.subreddit('loseit')
challenge_posts = loseit_sub.search("loseit challenge tracker", limit=1000)
topics_dict = { "title":[], "score":[], "id":[], "url":[], "comms_num": [],
                "created": [], "body":[]}

In [ ]:
for submission in challenge_posts:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

In [ ]:
topics_data = pd.DataFrame(topics_dict)

In [ ]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)
_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)

In [ ]:
topics_data.to_csv('loseit_search_history.csv')

### Now that we have searched through old loseit posts, we need to find the urls

In [ ]:
links = []
for body in topics_dict['body']:
    try:
        doc = etree.fromstring(markdown.markdown(re.sub('[\\n]', '', body)))
        for link in doc.xpath('//a'):
            web_url = link.get('href')
            if bool(re.search('spreadsheet',web_url)):
                links.append(web_url)
    except etree.XMLSyntaxError:
        print('incorrect code used in body')

In [ ]:
unique_spreadsheets = list(set(links))

### Now that we have the urls, we need to use the google API to download the data.

In [ ]:
# use creds to create a client to interact with the Google Drive API
names = []
for spreadsheet_link in unique_spreadsheets:
    scope = ['https://spreadsheets.google.com/feeds']
    creds = ServiceAccountCredentials.from_json_keyfile_name('loseit-sheets-6012c29a1f40.json', scope)
    gc = gspread.authorize(creds)
    sht = gc.open_by_url(spreadsheet_link)
    if bool(re.search('nter', sht.title)) == False and bool(re.search('/r/', sht.title)) == False and bool(re.search('Calculator', sht.title)) == False:
        sheet_name = re.sub('_\(responses\)' ,'', re.sub(',', '', re.sub('\]','', re.sub('\[','', re.sub(' ', '_', re.sub('  ', '_', re.sub('-', '', sht.title.lower())))))))
        if sheet_name not in names:
            print(f'sheet name: {sheet_name}, link: {spreadsheet_link}')
            names.append(sheet_name)
            try: 
                data_sheet = sht.worksheet('Tracker')
                data_vals = data_sheet.get_all_values()
                data_df = pd.DataFrame(data_vals[1:], columns=data_vals[0])
                data_df.to_csv('./data/raw_data/' + sheet_name + '.csv')
            except gspread.WorksheetNotFound:
                try:
                    data_sheet = sht.worksheet('Master Spreadsheet')
                    data_vals = data_sheet.get_all_values()
                    data_df = pd.DataFrame(data_vals[1:], columns=data_vals[0])
                    data_df.to_csv('./data/raw_data' + sheet_name + '.csv')
                except gspread.WorksheetNotFound:
                    print('')#sheet_name)
            else:
                print('')#sheet_name)

### Now that the data is downloaded, it is time to load in the data and clean it down to the catagories that we are interested in

In [143]:
weeks_col = {f'W{x}':f'Week {x}' for x in range(0, 11)}
new_names = {'W0 (SW)': 'Week 0', 'Sex': 'Gender', 'Male, Female, Other': 'Gender',
             'TEAM': 'Team', 'Teams': 'Team', 'Challenge GW': 'Challenge Goal Weight',
             'Challenge SW': 'Week 0', 'Signup weight': 'Week 0', 
             'MyFitnessPal Username/Link': 'MFP'}

In [181]:
df_list = []
p = Path('./data/raw_data/')
for challenge in p.rglob('*.csv'):
    test_df = pd.read_csv(challenge, index_col=0)
    test_df.rename(index=str, columns=new_names, inplace=True)
    test_df.columns = test_df.columns.str.strip().str.replace('?', '').str.replace(':','')
    # timestamp
    if 'Timestamp' not in test_df:
        test_df['Timestamp'] = np.NaN
    # Age
    test_df['Age'] = test_df[test_df.filter(regex=re.compile('Age', re.IGNORECASE)).columns[0]]
    # Gender
    if 'Gender' not in test_df:
        test_df['Gender'] = 'Unknown'
    # Ignore KGS
    test_df.drop(test_df.filter(regex=re.compile('kgs', re.IGNORECASE)).columns, axis=1, inplace=True)
    # Just Starting BMI
    test_df.drop(test_df.filter(regex=re.compile('BMI', re.IGNORECASE)).columns[1:], axis=1, inplace=True)
    # Username
    test_df.columns = test_df.columns.str.replace(test_df.filter(like='name').columns[0], 'Username')
    # Weigh-in Data
    test_df.rename(index=str, columns=weeks_col, inplace=True)
    test_df['Week 0'] = test_df[test_df.filter(regex=re.compile("current weight", re.IGNORECASE)).columns[0]]
    # Height
    test_df['Height'] = test_df[test_df.filter(regex=re.compile("Height", re.IGNORECASE)).columns[0]]    
    # Highest Weight
    if len(test_df.filter(regex=re.compile('Highest', re.IGNORECASE)).columns):
        test_df['Highest Weight'] = test_df[test_df.filter(regex=re.compile('Highest', re.IGNORECASE)).columns[0]]
    else:
        test_df['Highest Weight'] = np.NaN
    # Has NSV
    test_df['Has NSV'] = test_df[test_df.filter(regex=re.compile("NSV", re.IGNORECASE)).columns[0]].notnull().astype('int')
    # Goal Weight
    test_df['Challenge Goal Weight'] = test_df[test_df.filter(regex=re.compile("Goal Weight", re.IGNORECASE)).columns[0]]

    # Has a food tracker
    if len(test_df.filter(regex=re.compile('MyFitnessPal', re.IGNORECASE)).columns):
        test_df['MFP'] = test_df[test_df.filter(regex=re.compile('MyFitnessPal', re.IGNORECASE)).columns].notnull().astype('int')    
    test_df['Has MFP'] = test_df[test_df.filter(regex=re.compile("MFP", re.IGNORECASE)).columns[0]].notnull().astype('int')
    if len(test_df.filter(regex=re.compile("Loseit", re.IGNORECASE)).columns):
        test_df['Has Loseit'] = test_df[test_df.filter(regex=re.compile("Loseit", re.IGNORECASE)).columns].notnull().astype('int')
    else:
        test_df['Has Loseit'] = 0
    test_df['Has Food Tracker'] = test_df['Has MFP'] + test_df['Has Loseit']
    test_df[test_df['Has Food Tracker'] > 0] = 1
    
    # fitness tracker
    if len(test_df.filter(regex=re.compile('Fitbit', re.IGNORECASE)).columns):
        test_df['Has Activity Tracker'] = test_df[test_df.filter(regex=re.compile('Fitbit', re.IGNORECASE)).columns].notnull().astype('int')
    elif len(test_df.filter(regex=re.compile('Fitness tracker', re.IGNORECASE)).columns):
        test_df['Has Activity Tracker'] = test_df[test_df.filter(regex=re.compile('Fitness Tracker', re.IGNORECASE)).columns].notnull().astype('int')
    elif len(test_df.filter(regex=re.compile('Garmin', re.IGNORECASE)).columns):
        test_df['Has Activity Tracker'] = test_df[test_df.filter(regex=re.compile('Garmin', re.IGNORECASE)).columns].notnull().astype('int')
    elif len(test_df.filter(regex=re.compile('Strava', re.IGNORECASE)).columns):
        test_df['Has Activity Tracker'] = test_df[test_df.filter(regex=re.compile('Strava', re.IGNORECASE)).columns].notnull().astype('int')

    # clean up any possible duplicates
    test_df = test_df.loc[:,~test_df.columns.duplicated()]

    # Create the final Data Frame
    col_weeks = test_df.filter(regex=re.compile('Week', re.IGNORECASE)).columns.values
    col_names = ['Timestamp', 'Username', 'Team', 'Age', 'Gender', 'Height', 'Highest Weight',
                 'Challenge Goal Weight', 'Starting BMI', 'Has NSV', 'Has Food Tracker', 
                 'Has Activity Tracker']
    data_cols = col_names + list(col_weeks)
    data_df = test_df[data_cols]
    data_df.to_csv('./data/Cleaned_' + challenge.name)
    df_list.append((challenge.stem, data_df))

### Now that the data contains only what we are interested in learning, we need to fill in any missing values before we combine all of the challenges together

In [185]:
print(f'Challenge Name: {df_list[3][0]}\n')
print(df_list[3][1].columns.values)
df = df_list[3][1]

Challenge Name: new_challenge_new_year_new_goals_2018_edition

['Timestamp' 'Username' 'Team' 'Age' 'Gender' 'Height' 'Highest Weight'
 'Challenge Goal Weight' 'Starting BMI' 'Has NSV' 'Has Food Tracker'
 'Has Activity Tracker' 'Week 0' 'Week 1' 'Week 2' 'Week 3' 'Week 4'
 'Week 5' 'Week 6' 'Week 7']


In [188]:
df = df.dropna(subset=['Username', 'Challenge Goal Weight', df.columns.values[-1]])
df.loc[~df['Gender'].isin(['Female', 'Male', 'Unknown']), 'Gender'] = 'Other'
df.loc[pd.isnull(df['Highest Weight']), 'Highest Weight'] = df['Week 0']


In [189]:
df.count()

Timestamp                873
Username                 873
Team                     873
Age                      873
Gender                   873
Height                   873
Highest Weight           873
Challenge Goal Weight    873
Starting BMI             873
Has NSV                  873
Has Food Tracker         873
Has Activity Tracker     873
Week 0                   873
Week 1                   864
Week 2                   861
Week 3                   861
Week 4                   857
Week 5                   841
Week 6                   826
Week 7                   873
dtype: int64

### Now we want to fill in any other missing info

In [ ]:
p = Path('./data/')
for challenge in p.rglob('Cleaned*.csv'):
    df = pd.read_csv(challenge, index_col=0)
    print(challenge.stem, df.count())

In [41]:
test_df.loc[(test_df.shape[1] - 2) <= test_df.count(axis=1)]#.fillna(axis=1, method='ffill')
lb = test_df.filter(regex=re.compile("Height", re.IGNORECASE)).columns[0]
print(lb, lb in test_df)
test_df.columns.str.replace(test_df.columns.sort_values()[-1], 'Height')

Your Height(in inches) True


Index(['Timestamp', 'Username', 'Team', 'Age', 'Gender',
       'Current Weight(in pounds/lbs)', 'Challenge Goal Weight',
       'Your Height(in inches)', 'What is your current weight',
       'Challenge LBS Lost', 'LBS to Goal', 'Signup weight', 'Week 1',
       'Week 2', 'Week 3', 'Week 4', 'Week 5', 'Week 6', 'Highest Weight'],
      dtype='object')

In [42]:
test_df[test_df.columns.sort_values()[-1]]

1                          70
2                          69
4                          63
5                          61
6                          78
7                          59
8                          59
10                         63
11                         65
12                         71
13                         72
14                         65
15                         70
16                         66
17                         72
18                         62
19                         62
21                         69
22                         73
23                         66
24                         64
25                         67
27                         71
28                         66
29                         72
30                         64
32                         61
33                         69
35                         63
36                         74
                ...          
911                        66
914                        62
915       

In [ ]:
sns.catplot(x='Gender', kind="count", data=df)

In [ ]:
f_df = df[df['Gender']!='Male']
o_df = f_df[f_df.Gender != 'Female']
o_df

In [ ]:
sns.catplot(x='Gender', kind="count", data=df)

In [ ]:
test_df.columns

In [ ]:
f, ax = plt.subplots(1, 1)
ax = sns.distplot(test_df['Week 0'],label='starting weight')
ax = sns.distplot(test_df['Week 6'], label='final week')
ax.legend()